In [5]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16614, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 16614 (delta 51), reused 59 (delta 27), pack-reused 16522
Receiving objects: 100% (16614/16614), 15.15 MiB | 7.22 MiB/s, done.
Resolving deltas: 100% (11408/11408), done.


In [9]:
!cd yolov5 && pip3.10 install -r requirements.txt

In [16]:
!cd yolov5 && python3.10 train.py --img 416 --batch 64 --epochs 1 --data data.yaml --weights yolov5s.pt --workers 8

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.5 torch-2.3.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
# Loading custom trained YOLO Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp15/weights/last.pt', force_reload=True)
# Change path to whichevers latest run

In [ ]:
# Making custom dataset to test model
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib
from matplotlib.patches import Rectangle
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'


classes = ['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can', 'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']
label_map = {index: value for index, value in enumerate(classes)}
for i in label_map:
    print(str(i)+": "+label_map[i])


class TACODataset(Dataset):
    def __init__(self, type, target_transform=None):
        self.dir = "./data/" + type
        self.img_names = pd.read_csv(self.dir + "/img_names.txt")
        self.img_dir = self.dir + "/images/"
        self.target_transform = target_transform
        self.classes = ['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can', 'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']
        self.nclasses = 18

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.img_names.iloc[idx, 0] + ".jpg"
        image = read_image(img_path)
        label_file = open(self.dir + "/labels/" + self.img_names.iloc[idx, 0] + ".txt")
        label = label_file.read()

        if self.target_transform:
            label = self.target_transform(label)

        return image, label
    

def make_rect(label_info):
    coords = label_info.split()
    coords.pop(0)
    coords = [eval(i) for i in coords]
    coords = [i*416 for i in coords]
    rect = Rectangle((coords[0] - (coords[2]/2), coords[1] - (coords[3]/2)), coords[2], coords[3], linewidth=1, edgecolor='r', facecolor='none')
    return rect


def show_example(img, label):
    plt.imshow(img.permute(1, 2, 0))
    trash = label.split("\n")
    
    for t in trash:

        print(str(classes[int(t[0:2])]) + ": " + t)
        plt.gca().add_patch(make_rect(t))
    plt.show()

train_dataset = TACODataset("train")
validation_dataset = TACODataset("valid")
test_dataset = TACODataset("test")


show_example(*train_dataset[89])
show_example(*validation_dataset[40])
show_example(*test_dataset[0])

test_result = model(test_dataset[0][0])
test_result.print()
test_result.show()

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()